In [ ]:
# 크롤링 순서
# 1차 크롤링 : 블로그 조회 -> title / url / date
# 2차 크롤링 : url별 방문하여 해시태그 가져오기


- import

In [1]:
import os
import sys
import pandas as pd

import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

- url list 

In [3]:
# start_time = timer()

# 검색어 입력
f = open("data/station.txt", 'r', encoding='utf-8')
keywords = f.readlines()
keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []
for query in keywords:
    # start_crawling_time = timer()
    # print(start_crawling_time)
    try:
        # 검색 URL 생성
        base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(base_url)
        time.sleep(0.1)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        search_number = soup.find(class_="search_number")
        total_num = search_number.text.replace("건",'').replace(",",'')
        total_num = int(total_num)
        end_page=total_num//7+1

        # 페이지마다 게시물 가져오기
        for page_num in range(1, total_num // 7 + 2):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(0.1)  # 페이지가 로드될 때까지 대기

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 블로그 포스트 데이터 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            posts_date_data = soup.select('div.info_post > div.writer_info')

            # 타이틀 수가 7개가 아니면 루프를 빠져나오기
            if len(posts_data) >= 6:
                for post, post_date in zip(posts_data, posts_date_data):
                    title = post.text.replace('\n', '').strip()
                    href = post.attrs['href']
                    date = post_date.find('span', 'date').text.strip()
                    try :
                        date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                    except:
                        date_timestamp = endDate
                    crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
            else :
                break
    except Exception as e:
        print(f"에러 발생: {e}")

    # end_crawling_time = timer()
    # print(end_crawling_time)
    # print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))


crawling_blog_data = pd.DataFrame(crawling_blog_data)
file_name = f'data/crawling_blog_list_{startDate}-{endDate}.csv'
crawling_blog_data.to_csv(file_name, index=False)

- 해시태그 가져오기

In [26]:
# url list 가져오기
crawling_data = pd.read_csv(file_name)
crawling_data

# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


crawling_contents_data = []
url_lst = crawling_data['url']
for i in range(len(url_lst)) :
    url = str(url_lst[i])
    driver.get(url)

    title = crawling_data.loc[i,'title']
    date = crawling_data.loc[i,'posted_date']
    try :
        # iframe 접근
        WebDriverWait(driver,10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))

        # tag 찾아서 가져오기
        tags_element = driver.find_element(By.CLASS_NAME, "wrap_tag")
        tags = tags_element.text.strip('태그').replace('\n','')
        # overlays_tag = "wrap_tag"
        # html_tags = driver.find_element(By.CSS_SELECTOR, overlays_tag)
        # hash_tags = html_tags.text
        crawling_contents_data.append({"location":query, "title":title, "url":url, "tags":tags, "posted_date":date})
    except Exception as e:
        print(f"error ({i}) : {str(e)}")

driver.quit()

- 파일저장

In [ ]:
df_tags = pd.DataFrame(crawling_contents_data)
df_tags

In [30]:
file_name = f'data/crawling_blog_data_{startDate}-{endDate}.csv'
df_tags.to_csv(f".csv", index=False)

- test

In [4]:
# start_time = timer()

# 검색어 입력
# f = open("crawling/station.txt", 'r', encoding='utf-8')
# keywords = f.readlines()
# keywords_data = []
query= input()

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []

# start_crawling_time = timer()
# print(start_crawling_time)
try:
    # 검색 URL 생성
    base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(base_url)
    time.sleep(0.1)  # 페이지가 로드될 때까지 대기

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    search_number = soup.find(class_="search_number")
    total_num = search_number.text.replace("건",'').replace(",",'')
    total_num = int(total_num)
    end_page=total_num//7+1

    # 페이지마다 게시물 가져오기
    for page_num in range(1, total_num // 7 + 2):
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.1)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 블로그 포스트 데이터 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        posts_date_data = soup.select('div.info_post > div.writer_info')

        # 타이틀 수가 7개가 아니면 루프를 빠져나오기
        if len(posts_data) >= 6:
            for post, post_date in zip(posts_data, posts_date_data):
                title = post.text.replace('\n', '').strip()
                href = post.attrs['href']
                date = post_date.find('span', 'date').text.strip()
                try :
                    date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                except:
                    date_timestamp = endDate
                crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
        else :
            break
except Exception as e:
    print(f"에러 발생: {e}")

# end_crawling_time = timer()
# print(end_crawling_time)
# print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))

In [16]:
df = pd.DataFrame(crawling_blog_data)
df.sort_values("posted_date", axis = 0, ascending=False, inplace=True)
file_name = f"data/naver_blog_list_{startDate}-{endDate}.csv"
df.to_csv(file_name, index=False)

In [17]:
blog_lst_data = pd.read_csv(file_name)
blog_lst_data

,location,title,url,posted_date
0,회기역,"청량리 근처 숙소, 회기역 호텔 : 호텔 모우",https://blog.naver.com/kkyeong1203/223457318959,2024-05-25
1,회기역,회기역 카페 경희대 브런치 데이트 레프트커피,https://blog.naver.com/fusgodl/223457172586,2024-05-25
2,회기역,회기역 인근 서울 숨은 크루키 맛집/버터스카치가 맛있는 집[카페 마이 달리아],https://blog.naver.com/leeyougin/223457616243,2024-05-25
3,회기역,경희대 서울캠퍼스 회기역 크루키 구움과자 맛집 [마이 달리아],https://blog.naver.com/bread0810/223457604621,2024-05-25
4,회기역,서울 회기역 경희대 근처 점심 맛집 언니네 함바그 내돈내산,https://blog.naver.com/richenoughforfence/2234...,2024-05-25
...,...,...,...,...
58,회기역,"회기역 카페 :: 한시십일분, 시립대 경희대 근처 카공 디저트 맛집!",https://blog.naver.com/jinjini7/223451417960,2024-05-19
59,회기역,"회기역 치과 치아 깨짐 일어난 환자,신경치료 후 크라운 과정으로 치아 보존 성공한 사례",https://blog.naver.com/miteumdent/223449064971,2024-05-18
60,회기역,윤양식_회기역 파스타 맛집,https://blog.naver.com/duoeun01/223450289075,2024-05-18
61,회기역,회기 맛집 / 회기역 파스타 / 회기 레스토랑 / 경희대 시립대 파스타 맛집 ‘윤양식’,https://blog.naver.com/tmddus8741/223450812660,2024-05-18


In [25]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_argument('headless')
driver = webdriver.Chrome(options = options)


crawling_contents_data = []

url_lst = blog_lst_data['url']
for i in range(len(url_lst)) :
    url = str(url_lst[i])
    driver.get(url)

    try :
        # iframe 접근
        WebDriverWait(driver,10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))

        # tag 찾아서 가져오기
        tags_element = driver.find_element(By.CLASS_NAME, "wrap_tag")
        tags = tags_element.text.strip('태그').replace('\n','')
        # overlays_tag = "wrap_tag"
        # html_tags = driver.find_element(By.CSS_SELECTOR, overlays_tag)
        # hash_tags = html_tags.text
        crawling_contents_data.append({"location":query, "url":url, "tags":tags})
    except Exception as e:
        print("error ({i}) : {str(e)}")

driver.quit()


In [21]:
print(crawling_contents_data)

[{'url': 'https://blog.naver.com/kkyeong1203/223457318959', 'tags': '#동대문숙소#청량리근처숙소#회기역호텔#호텔모우#혼자쉬기좋은숙소#서울여행#숙소후기#외대앞역숙소#회기역숙소#청량리숙소'}, {'url': 'https://blog.naver.com/fusgodl/223457172586', 'tags': '#회기역카페#회기카페#서울경희대카페#회기브런치#회기데이트#레프트커피#회기레프트커피#레프트커피회기#경희대브런치#회기브런치카페#경희대브런치카페'}, {'url': 'https://blog.naver.com/leeyougin/223457616243', 'tags': '#회기역인근서울숨은크루키맛집#버터스카치맛집#회기역인근마들렌맛집#마들렌잘하는집#회기역카페#회기디저트맛집#커피맛집#회기카페#청량초등학교카페'}, {'url': 'https://blog.naver.com/bread0810/223457604621', 'tags': '#크루키#크루키맛집#서울카페#동대문구카페#회기카페#회기역카페#경희대카페#경희대학교서울캠퍼스#회기크루키#경희대크루키#버터스카치#마들렌#구움과자맛집#서울카페추천#일상#데일리#cafe#crookie'}, {'url': 'https://blog.naver.com/richenoughforfence/223457183147', 'tags': '#경희대맛집#경희대맛집추천#회기역맛집#회기맛집#언니네함바그#함박스테이크#함박스테이크맛집#경희대점심#경희대점심맛집#경희대혼밥#경희대학교맛집#경희대정문맛집#경희대후문맛집#경희대근처맛집#회기역근처맛집#회기역혼밥#회기혼밥'}, {'url': 'https://blog.naver.com/verrrano/223456754270', 'tags': '#맛집#서울맛집#회기역#회기맛집#경희대맛집#경희대#경희대축제#경희대학교#보쌈#족발#회기왕족발#회기왕족발보쌈#회식#회식메뉴#회식추천#단체모임#단체모임장소#회기술집#회기카페#회기역맛집#서이추#서이추환영#블로그서이추'}, {'url': 'http

In [28]:
df_tags = pd.DataFrame(crawling_contents_data)
df_tags

,location,url,tags
0,회기역,https://blog.naver.com/kkyeong1203/223457318959,#동대문숙소#청량리근처숙소#회기역호텔#호텔모우#혼자쉬기좋은숙소#서울여행#숙소후기#외...
1,회기역,https://blog.naver.com/fusgodl/223457172586,#회기역카페#회기카페#서울경희대카페#회기브런치#회기데이트#레프트커피#회기레프트커피#...
2,회기역,https://blog.naver.com/leeyougin/223457616243,#회기역인근서울숨은크루키맛집#버터스카치맛집#회기역인근마들렌맛집#마들렌잘하는집#회기역...
3,회기역,https://blog.naver.com/bread0810/223457604621,#크루키#크루키맛집#서울카페#동대문구카페#회기카페#회기역카페#경희대카페#경희대학교서...
4,회기역,https://blog.naver.com/richenoughforfence/2234...,#경희대맛집#경희대맛집추천#회기역맛집#회기맛집#언니네함바그#함박스테이크#함박스테이크...
...,...,...,...
58,회기역,https://blog.naver.com/jinjini7/223451417960,#회기카페#경희대카페#시립대카페#회기역카페#한시십일분#회기카페추천#이웃#이웃환영#서...
59,회기역,https://blog.naver.com/miteumdent/223449064971,#11#청량리#청량리역#청량리치과#청량리역치과#청량리맛집#치아신경치료#신경치료과정#...
60,회기역,https://blog.naver.com/duoeun01/223450289075,#회기역파스타#회기역맛집#경희대맛집#경희대파스타#회기레스토랑#시립대맛집#시립대파스타...
61,회기역,https://blog.naver.com/tmddus8741/223450812660,#회기파스타#회기양식#경희대파스타#시립대파스타#회기파스타맛집#회기양식맛집#경희대양식...


In [27]:
df_tags.to_csv("tags_testg(회기역).csv", index=False)